In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

%pip install -q spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
%pip install -q pytorch-metric-learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as transforms
import cv2
import torchvision.models as models
from pytorch_metric_learning import losses
import torch.optim as optim
from torch.optim import lr_scheduler
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
class Config:
  spurious_correlation_strength  = 0.995
  spurious_feature_difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE
  label_noise = 0.001
  size = 28
  flip_probability = 0.5
  batch_size = 256
  backbone_pretrained = False
  projection_dim = 128
  T_max = 10
  lr = 5e-4
  min_lr = 1e-5
  weight_decay = 1e-6
  temperature = 0.5

## Dataset

In [6]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="train",
    label_noise=Config.label_noise
)
trainset.initialize()

valset = SpuCoMNIST(
    root="/data/mnist/",

    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="test",
    label_noise=Config.label_noise
)
valset.initialize()


100%|██████████| 9912422/9912422 [00:00<00:00, 116205656.72it/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 30375048.60it/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 41872367.17it/s]

Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 5057214.96it/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



100%|██████████| 10000/10000 [00:01<00:00, 7383.26it/s]


In [7]:

## Augmentation
from torchvision.transforms import v2

size = 28
train_transform = transforms.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    v2.RandomGrayscale(p=0.2),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    v2.ToImage(),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [8]:
class SpucoDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img = self.dataset[index][0]
        label = self.dataset[index][1]

        if self.transform is None:
          return img, label

        augmented_img = self.transform(img)
        return (img, augmented_img), label

In [9]:
train_dataset = SpucoDataset(trainset, train_transform)
val_dataset = SpucoDataset(valset, test_transform)

dataloaders = {'train': DataLoader(dataset=train_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=True),
         'eval': DataLoader(dataset=val_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=False)}

In [10]:
from IPython.display import display
for (k, v) in valset.group_partition.items():
  print(f"Group: {k}, Qty: {len(v)}")
  img = T.ToPILImage()(valset[v[0]][0])
  display(img)

Group: (3, 2), Qty: 397


Group: (1, 3), Qty: 408


Group: (0, 4), Qty: 423


Group: (0, 0), Qty: 423


Group: (2, 1), Qty: 375


...

Group: (2, 0), Qty: 375


Group: (4, 0), Qty: 397


Group: (1, 2), Qty: 408


Group: (1, 1), Qty: 409


Group: (4, 4), Qty: 396


In [11]:
T.ToPILImage()(valset[v[0]][0])

## Contrastive Loss

In [12]:
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#ntxentloss
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#selfsupervisedloss
criterion = losses.NTXentLoss(temperature=Config.temperature)
criterion = losses.SelfSupervisedLoss(criterion)

## Model

In [13]:
## https://github.com/berfukaraca/SSL-Cifar10-ContrastiveLearning/blob/main/colab_notebooks/SSL_SimCLR_Cifar10_CL.ipynb

class SimCLR_model(nn.Module):
    def __init__(self, base_encoder, projection_dim, n_features):
        super(SimCLR_model, self).__init__()
        self.n_features = n_features
        self.base_encoder = base_encoder

        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_dim, bias=False),
            nn.BatchNorm1d(projection_dim),
        )

    def forward(self, x_i):
        out = self.projectionHead(self.base_encoder(x_i))

        return out

In [14]:
## model and optimizer
model = models.resnet50()
# print(model)
clr_model = SimCLR_model(model, Config.projection_dim, 1000) ## output dimension of resnet18 is 1000
optimizer = optim.Adam(clr_model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
## cosine annealing lr: https://arxiv.org/pdf/1608.03983.pdf
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr)

clr_model.to(device)

SimCLR_model(
  (base_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


In [15]:
import os
def train(epoch):
    print('\nEpoch: %d' % epoch)
    clr_model.train()
    s=0
    tot_loss = 0
    total = 0
    batch_idx=0
    i=0
    batch_num =1
    val_loss = 0
    for batch_idx, (images, _) in enumerate(dataloaders['train']):
        batch_tot_loss = 0
        if (batch_num % 50 == 1):
          print("in batch", batch_num)
        batch_num=batch_num+1
        images, augmented_images = images[0], images[1]
        images, augmented_images = images.to(device), augmented_images.to(device)
        optimizer.zero_grad()

        images_emb = clr_model(images)
        augmented_images_emb = clr_model(augmented_images)
        loss = criterion(images_emb, augmented_images_emb)

        loss.backward()

        optimizer.step()
        tot_loss += loss.item()
        batch_tot_loss += loss.item()
        if (batch_idx % 200 == 1):
            print('batch loss=', batch_tot_loss)
    scheduler.step()



    # Validation
    model.eval()
    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(dataloaders['eval']):
            val_batch_loss = 0
            if (batch_num % 50 == 1):
                print("in batch", batch_num)
            batch_num=batch_num+1
            images, augmented_images = images[0], images[1]
            images, augmented_images = images.to(device), augmented_images.to(device)


            images_emb = clr_model(images)
            augmented_images_emb = clr_model(augmented_images)
            loss = criterion(images_emb, augmented_images_emb)



            val_loss += loss.item()
            val_batch_loss += loss.item()
        if (batch_idx % 200 == 1):
          print('batch loss=', val_batch_loss)

    print(f"epoch {epoch}train loss= {tot_loss/len(dataloaders['train'])}")
    print(f"epoch {epoch}val loss= {val_loss/len(dataloaders['train'])}")

    if epoch % 10 ==0:
        save_dir = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
        if not os.path.isdir(save_dir):
          os.mkdir(save_dir)
        modelstate_save_name = f'modelstate_bs_restart_r_{Config.spurious_correlation_strength}_LARGE_epoch{epoch}.pt'
        path_modelstate = os.path.join(save_dir, modelstate_save_name)
        torch.save(clr_model.state_dict(), path_modelstate)

        parameters_end = {
            'net': clr_model.state_dict(),
            'epoch': epoch,
            'loss': tot_loss
        }
        paramsend_save_name = f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_LARGE.pt'
        path_paramsend = os.path.join(save_dir, paramsend_save_name)
        torch.save(parameters_end, path_paramsend)



In [16]:
start_epoch=1
max_epoch=30

for epoch in range(start_epoch, start_epoch+max_epoch):
  train(epoch)


Epoch: 1


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 6.28194522857666
in batch 51
in batch 101
in batch 151
in batch 201
epoch 1train loss= 5.29264822919318
epoch 1val loss= 1.2059463795195235

Epoch: 2


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 5.077808856964111
in batch 51
in batch 101
in batch 151
in batch 201
epoch 2train loss= 5.0677802232985805
epoch 2val loss= 1.1879163295664685

Epoch: 3


...

in batch 1
batch loss= 4.623539447784424
in batch 51
in batch 101
in batch 151
in batch 201
epoch 28train loss= 4.657310262639472
epoch 28val loss= 1.0828243823761636

Epoch: 29


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 4.644150257110596
in batch 51
in batch 101
in batch 151
in batch 201
epoch 29train loss= 4.647776091352422
epoch 29val loss= 1.0700544486654566

Epoch: 30


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

in batch 1
batch loss= 4.654287338256836
in batch 51
in batch 101
in batch 151
in batch 201
epoch 30train loss= 4.642476760326548
epoch 30val loss= 1.0739763721506645


In [17]:
## Load the pre-trained model parameters
paramsend_save_name = f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_LARGE.pt'
save_dir = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
path_paramsend = path_paramsend = os.path.join(save_dir, paramsend_save_name)
checkpoint = torch.load(path_paramsend, map_location = torch.device(device))
loss= checkpoint['loss']
epoch = checkpoint['epoch']

start_epoch=epoch+1
clr_model.load_state_dict(checkpoint['net'])

<All keys matched successfully>

## PCA and 2D Scatter Plot

In [18]:
torch.cuda.empty_cache()

In [19]:
## https://www.kaggle.com/code/minc33/visualizing-high-dimensional-clusters

import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA


torch.cuda.empty_cache()
device = 'cpu'
clr_model.to(device)

embeddings = []
targets = []

for batch_idx, ((images, _), labels) in enumerate(dataloaders['eval']):
  embeddings.append(clr_model(images).detach().cpu())
  targets.append(labels)
  torch.cuda.empty_cache()


# clr_model = None

embeddings = torch.cat(embeddings).detach().cpu().numpy()
targets = torch.cat(targets).cpu().numpy()

In [20]:
import pandas as pd
comp = 100
pca_2d = PCA(n_components=comp)
PCs_2d = pd.DataFrame(pca_2d.fit_transform(embeddings))
PCs_2d.columns = [f"PC{i+ 1}" for i in range(comp)]
pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')

In [21]:
pce = pca_2d.fit_transform(embeddings)

In [22]:
pca_2d.singular_values_

array([431.29135  , 376.7262   , 365.97794  , 358.33484  , 327.7286   ,
       306.94836  , 291.31088  , 283.60336  , 265.84268  , 230.11581  ,
       224.09875  , 200.9585   , 179.40945  , 157.27893  , 144.0492   ,
        64.21597  ,  31.184158 ,  21.407595 ,  19.309366 ,  14.304529 ,
        11.365482 ,  11.254357 ,  10.34032  ,  10.004988 ,   9.775862 ,
         9.599835 ,   9.509261 ,   9.258687 ,   8.9939785,   8.675992 ,
         8.4806795,   8.424311 ,   8.246831 ,   8.162687 ,   7.861255 ,
         7.71981  ,   7.6705627,   7.4964767,   7.3925385,   7.2442045,
         7.1367197,   7.000287 ,   6.8397   ,   6.745452 ,   6.510745 ,
         6.4316206,   6.358011 ,   6.2222524,   6.1792965,   6.0999894,
         5.9863725,   5.8274775,   5.74942  ,   5.6914797,   5.5908623,
         5.4977436,   5.420986 ,   5.2663293,   5.242442 ,   5.1365967,
         5.109619 ,   5.019848 ,   4.980899 ,   4.864389 ,   4.807731 ,
         4.7472625,   4.6524234,   4.6085515,   4.5703673,   4.4

In [23]:
pca_embs['true_label'] = targets

In [24]:
clusters = {k : np.stack([PCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [25]:
import plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

traces = []

for (k, v) in sorted(clusters.items()):
  traces.append(go.Scatter(
                    x = v[:, 2],
                    y = v[:, 3],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))
data = traces

title = "Visualizing Clusters in Two Dimensions Using PCA"

layout = dict(title = title,
              xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)

In [26]:
for (true, spur), idxs in valset.group_partition.items():
  for i in idxs:
    pca_embs.loc[i, 'spurious_label'] = spur
pca_embs['spurious_label'] = pca_embs['spurious_label'].astype(int)

In [27]:
valset.group_partition.keys()

dict_keys([(3, 2), (1, 3), (0, 4), (0, 0), (2, 1), (0, 1), (4, 3), (2, 2), (0, 2), (4, 2), (0, 3), (2, 4), (3, 0), (4, 1), (3, 4), (3, 1), (2, 3), (1, 0), (3, 3), (1, 4), (2, 0), (4, 0), (1, 2), (1, 1), (4, 4)])

In [28]:
true_label_clusters = {
    # True label: {group, cluster_dim}
}
for l in pca_embs['true_label'].unique():
  true_cluster = pca_embs[pca_embs['true_label'] == l]
  true_emb = true_cluster.iloc[:, :128].values
  pca = PCA(n_components=3)
  PCs = pd.DataFrame(pca.fit_transform(embeddings), columns = ['PC1_TL', 'PC2_TL', 'PC3_TL'])
  PCs = pd.concat([PCs,true_cluster[['true_label', 'spurious_label']]], axis=1, join='inner')
  true_label_clusters[l] = PCs
  # pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')


In [29]:
spurious_label_clusters = {
    # Spurious label: {group, cluster_dim}
}
for l in pca_embs['spurious_label'].unique():
  spurious_cluster = pca_embs[pca_embs['spurious_label'] == l]
  spurious_emb = spurious_cluster.iloc[:, :128].values
  pca = PCA(n_components=3)
  PCs = pd.DataFrame(pca.fit_transform(embeddings), columns = ['PC1_TL', 'PC2_TL', 'PC3_TL'])
  PCs = pd.concat([PCs,spurious_cluster[['true_label', 'spurious_label']]], axis=1, join='inner')
  spurious_label_clusters[l] = PCs
  # pca_embs = pd.concat([pd.DataFrame(embeddings),PCs_2d], axis=1, join='inner')


In [30]:
import os

save_dir = f"/content/drive/MyDrive/CS260D_final_project/plots/"
for k, v in true_label_clusters.items():
  traces = []
  for s in np.sort(v['spurious_label'].unique()):
    group = v[v['spurious_label'] == s]
    traces.append(go.Scatter(
                      x = group['PC2_TL'],
                      y = group['PC3_TL'],
                      #z = group['PC3_TL'],
                      mode = "markers",
                      name = f"Subgroup {s}",
                      text = None))
  data = traces

  title = f"Visualizing Clusters in Two Dimensions Using PCA - True Label {k}"

  layout = dict(title = title,
                xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
                yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
              )

  fig = dict(data = data, layout = layout)
  save_name = f"simCLR_{Config.spurious_correlation_strength}_LARGE_TL{k}_PCA_plot.html"
  if os.path.isdir(save_dir):
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))
  else:
    os.mkdir(save_dir)
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))

  iplot(fig)

In [31]:
import os

save_dir = f"/content/drive/MyDrive/CS260D_final_project/plots/"
for k, v in spurious_label_clusters.items():
  traces = []
  for s in np.sort(v['true_label'].unique()):
    group = v[v['true_label'] == s]
    traces.append(go.Scatter(
                      x = group['PC1_TL'],
                      y = group['PC3_TL'],
                      #z = group['PC3_TL'],
                      mode = "markers",
                      name = f"True Label {s}",
                      text = None))
  data = traces

  title = f"Visualizing Clusters in Two Dimensions Using PCA - Spurious Label {k}"

  layout = dict(title = title,
                xaxis= dict(title= 'PC1',ticklen= 5,zeroline= False),
                yaxis= dict(title= 'PC2',ticklen= 5,zeroline= False)
              )

  fig = dict(data = data, layout = layout)
  save_name = f"simCLR_{Config.spurious_correlation_strength}_LARGE_TL{k}_Spurious_Label_PCA_plot.html"
  if os.path.isdir(save_dir):
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))
  else:
    os.mkdir(save_dir)
    go.Figure.write_html(fig, os.path.join(save_dir, save_name))

  iplot(fig)

## t-SNE

In [32]:
from sklearn.manifold import TSNE

perplexity = 50

#T-SNE with two dimensions
tsne_2d = TSNE(n_components=2, perplexity=perplexity)

TCs_2d = pd.DataFrame(tsne_2d.fit_transform(embeddings))
TCs_2d.columns = ["TC1_2d","TC2_2d"]
tsne_embs = pd.concat([pd.DataFrame(embeddings),TCs_2d], axis=1, join='inner')

In [33]:
tsne_embs

,0,1,2,3,4,5,6,7,8,9,...,120,121,122,123,124,125,126,127,TC1_2d,TC2_2d
0,-0.488330,-0.813822,0.718462,1.086207,0.147801,-1.289257,-0.436500,-0.212963,0.990836,0.436883,...,0.932292,-0.670967,0.196100,-0.084355,0.675138,-0.279659,-1.101647,-0.207140,28.238964,13.708904
1,-0.349727,-1.018899,0.673348,0.172039,-0.158273,-0.450911,-0.199468,-0.158542,0.272262,0.190251,...,0.780011,1.308016,0.817013,-0.579103,0.649417,0.579464,-0.096533,-0.223745,-23.271011,-15.696385
2,-1.145260,-0.741233,-0.677386,0.136290,0.571414,1.027354,-0.966206,-1.825762,-1.433244,-2.211956,...,-1.663195,-0.822356,-1.394597,-0.152992,-0.107218,-0.471051,-0.787181,0.608470,42.401058,-45.452156
3,0.826484,-0.387631,0.505480,-0.739737,-1.886126,2.778145,-0.697979,-0.736552,1.822273,1.134123,...,1.172296,-2.227422,0.838299,0.335380,0.207307,0.592672,1.796219,2.266689,-28.519680,63.216122
4,-0.682950,-0.644378,0.735636,0.423820,-0.010276,-0.002959,0.610378,0.350574,-0.309086,-0.053672,...,0.298045,1.348098,0.994173,-0.484580,0.478404,0.868382,0.849697,0.475956,-36.434425,-36.085144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.519532,-1.162672,1.063139,-0.336687,0.695449,0.525435,0.680721,0.590743,0.133323,-0.339712,...,0.307695,2.230146,1.215256,-1.370901,0.940965,0.482398,1.028957,-1.284793,-36.035305,-15.232913
9996,0.697483,0.094752,-2.098461,-1.566411,-1.534325,-1.970140,-1.467783,0.412943,1.498119,0.927351,...,1.159520,-0.026135,-0.462708,0.432997,0.132491,-1.184659,-0.075052,1.153983,15.127049,44.101917
9997,2.291490,-1.137508,1.378979,0.266760,1.439231,0.768220,0.987116,-0.418171,-0.947234,-0.735319,...,0.374150,-0.773640,-1.831832,-0.190796,0.870957,1.367230,0.665239,-0.973089,40.258781,-22.952734
9998,0.476359,0.076729,-0.543529,-0.067659,0.864876,-0.604218,0.374383,0.643789,-0.110315,0.611758,...,1.172673,-0.420808,-0.051387,-1.668121,-0.193717,-1.499419,0.627225,0.247945,-11.802580,34.393677


In [34]:
tsne_clusters = {k : np.stack([TCs_2d.loc[i].values for i in v]) for (k, v) in valset.group_partition.items()}

In [35]:
tsne_traces = []

for (k, v) in sorted(tsne_clusters.items()):
  tsne_traces.append(go.Scatter(
                    x = v[:, 0],
                    y = v[:, 1],
                    mode = "markers",
                    name = "True label {}, Subgroup {}".format(k[0], k[1]),
                    text = None))

In [36]:
data = tsne_traces

title = "Visualizing Clusters in Two Dimensions Using t-SNE"

layout = dict(title = title,
              xaxis= dict(title= 'TC1',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'TC2',ticklen= 5,zeroline= False)
             )

fig = dict(data = data, layout = layout)

iplot(fig)